<a href="https://colab.research.google.com/github/8itoonmaria/IAT360_FINAL_PROJECT/blob/main/IAT360_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Necessary Libraries

---


In [1]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

## LOAD LLM

In [2]:
# Load a pre-trained language model for text generation
llm = pipeline('text-generation', model='gpt2')  # A lightweight example for demonstration

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


## Prepare a Dataset

In [3]:
from datasets import load_dataset

documents = load_dataset("climatebert/environmental_claims")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-98aa5228a06a17(…):   0%|          | 0.00/215k [00:00<?, ?B/s]

data/validation-00000-of-00001-2553e47d4(…):   0%|          | 0.00/28.9k [00:00<?, ?B/s]

data/test-00000-of-00001-79fd931297fff76(…):   0%|          | 0.00/28.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2117 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/265 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/265 [00:00<?, ? examples/s]

## Generate Dense Embeddings and Build an Index

In [5]:
# Step 1: Create dense embeddings for the documents
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # A lightweight model for embedding
embeddings = embedding_model.encode(documents['train'])  # Generate embeddings for the 'train' split

# Step 2: Create a pandas DataFrame for indexing
# Combine documents and embeddings in a DataFrame
df = pd.DataFrame({
    "Document": documents['train']['text'], # Access 'text' column from the 'train' split
    "Embedding": list(embeddings)  # Store embeddings as lists for easier handling
})

# Display the DataFrame
print("Indexed Documents:")
(df.head())


Indexed Documents:


,Document,Embedding
0,The project will make a significant contributi...,"[-0.0799904, 0.11814675, 0.0041972417, -0.0147..."
1,"So in -- first of all, the segments, we are --...","[0.052529894, -0.040122908, 0.06840219, -0.023..."
2,Building a smarter and stronger energy grid th...,"[-0.087968916, 0.049407203, -0.008678998, 0.05..."
3,"Generally, first of all our Transmission depar...","[-0.0079747215, -0.0198657, 0.04407607, 0.0360..."
4,"But for the most part, our challenge is showin...","[0.06293592, -0.0009297496, -0.0054724347, 0.0..."


In [7]:
def retrieve_with_pandas(query, top_k=2):
    # Generate embedding for the query
    query_embedding = embedding_model.encode([query])[0]

    # Compute similarity scores (cosine similarity)
    df['Similarity'] = df['Embedding'].apply(lambda x: np.dot(query_embedding, x) /
                                             (np.linalg.norm(query_embedding) * np.linalg.norm(x)))
    # Sort by similarity and return top-k results
    results = df.sort_values(by="Similarity", ascending=False).head(top_k)
    return results[["Document", "Similarity"]]



# Define a function to use RAG
def generate_with_rag(query):
    # Retrieve context
    context = retrieve_with_pandas(query)["Document"]
    # Combine context with query
    input_text = f"Respond the query based on the context. Context: {context} Query: {query}"
    # Use LLM to generate a response
    response = llm(input_text, max_length=150, num_return_sequences=1)
    return response[0]['generated_text']

In [8]:
query = "is gas bad for the environment?"
results = retrieve_with_pandas(query, top_k=2)

print("\nQuery:", query)
print("\nTop Retrieved Documents:")
print(results)


Query: is gas bad for the environment?

Top Retrieved Documents:
                                               Document  Similarity
880   Gas is also a cleaner fuel with resultant envi...    0.667875
1196  An early analysis of carbon and water in our s...    0.572975


In [9]:
# Ask the same question with RAG
question = "is gas bad for the environment?"
response_rag = generate_with_rag(question)
print("****")
print("Response from LLM with RAG:", response_rag)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


****
Response from LLM with RAG: Respond the query based on the context. Context: 880     Gas is also a cleaner fuel with resultant envi...
1196    An early analysis of carbon and water in our s...
Name: Document, dtype: object Query: is gas bad for the environment? Is it a good idea to use it? Is it a good idea to use it?
1227     The document is a good idea. This is a good thing.
1228     Is the document good?
1229     And is it good?
1230      That was a good question. Is that better than going back to the page?
1231      The document is a good idea. This is a good thing.
1232        A good idea is to create a document that is good before it's deployed. A good idea is to create a document that is good before it's deployed.
1233                                                             
1234                                        
